# Model Response

## Imports

In [ ]:
import dolfin
import numpy
import math
import dolfin_mech     as dmech
import micro_poro_structure_generator as gen

## Defining geometry

In [ ]:
seeds_filename = "Test.dat"
mesh_filebasename = "Test-mesh"

domain_y = 0.1 * 0.8
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.012 * 0.8

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = - 0.023 * 0.8,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)
dV = dolfin.Measure("dx",domain=mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])

V = (xmax - xmin)*(ymax - ymin)
VS0 = dolfin.assemble(dolfin.Constant(1) * dV)
Vf0 = V - VS0

## Loading 

In [ ]:
qois_filename = "Test-qois.dat"
res_basename = "Test"

params_dimless = [0, 0,  0, 0]
params_initial = [0.08855929243285596, 0.011039510924095856, 0.6281487879627474, 3.409513378002055]

par = [params_initial[0]*(1 + params_dimless[0]), params_initial[1]*(1 + params_dimless[1]), params_initial[2]*(1 + params_dimless[2]), params_initial[3]*(1 + params_dimless[3])]
# graph_printer(smith_gamma_0_inf, par)

mat_params = {"model":"exponentialneoHookean", "parameters":{"beta1":par[0], "beta2":par[1], "beta3":par[2], "beta4":100*par[0], "alpha":par[3]}} 

pf= 2.5
load_params = {}
load_params["pf"] = pf
load_params["sigma_bar_00"] = 0.0
load_params["sigma_bar_11"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["gamma"] = 0
# load_params["gamma"] = 15e-3

## Model response

In [ ]:
problem = dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.005, "dt_max":0.1},
    res_basename=res_basename,
    write_qois_limited_precision=False,
    verbose=1
)

### Surface dependent surface tension

#### Inflation

In [ ]:
load_params = {}
load_params["pf_lst"] = [0.1, 0.5]
load_params["sigma_bar_00_lst"] = [0.0, 0.0]
load_params["sigma_bar_11_lst"] = [0.0, 0.0]
load_params["sigma_bar_01_lst"] = [0.0, 0.0]
load_params["sigma_bar_10_lst"] = [0.0, 0.0]
load_params["gamma_lst"] = [0.03, 0.03]
load_params["tension_params"] = {"surface_dependancy":1, "d1":1.0130287663205635, "d2":1.6742366271475184, "d3":-10.288589574038403}

step_params = {}
step_params["n_steps"] = 2
step_params["Deltat"] = 1.
step_params["dt_ini"] = 0.01
step_params["dt_min"] = 0.001
step_params["dt_max"] = 0.005

phi = dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params=step_params,
    res_basename=res_basename,
    write_qois_limited_precision=False,
    verbose=1
)

#### Deflation

In [ ]:
load_params = {}
load_params["pf_lst"] = [0.1, 0.5]
load_params["sigma_bar_00_lst"] = [0.0, 0.0]
load_params["sigma_bar_11_lst"] = [0.0, 0.0]
load_params["sigma_bar_01_lst"] = [0.0, 0.0]
load_params["sigma_bar_10_lst"] = [0.0, 0.0]
load_params["gamma_lst"] = [0.03, 0.03]
load_params["tension_params"] = {"surface_dependancy":1, "d1":1.030747711797792, "d2":2.1708848554526874, "d3":-14.828598856766776}

step_params = {}
step_params["n_steps"] = 2
step_params["Deltat"] = 1.
step_params["dt_ini"] = 0.1
step_params["dt_min"] = 0.01
step_params["dt_max"] = 0.05

phi = dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params=step_params,
    res_basename=res_basename,
    write_qois_limited_precision=False,
    verbose=1
)

## Post processing

In [ ]:
for operator in problem.operators: 
    if hasattr(operator, "material"):
        material = operator.material
        break

In [ ]:
# Macroscopic strain
print("U_bar_x:" +str(problem.get_macroscopic_stretch_subsol().func.vector().get_local()[0]))
print("U_bar_y:" +str(problem.get_macroscopic_stretch_subsol().func.vector().get_local()[3]))
print("U_bar_xy:" +str(problem.get_macroscopic_stretch_subsol().func.vector().get_local()[1]))

# Kinematics
U_bar = problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))
F_bar = U_bar + numpy.eye(2)
C_bar = F_bar.T * F_bar
E_bar = 0.5*(C_bar - numpy.eye(2))
J_bar = numpy.linalg.det(F_bar)
FmTN = dolfin.dot(dolfin.inv(problem.kinematics.F).T, problem.mesh_normals)

print("J: "+str(J_bar))


# Poromechanics propoerties
T = dolfin.sqrt(dolfin.inner(FmTN, FmTN))
S = dolfin.assemble(T * problem.kinematics.J * problem.dS(0))
S0 = dolfin.assemble(dolfin.Constant(1)*problem.dS(0))


print("S:" +str(S))
print("S*: " + str(S/S0))
print("IC: "+str(numpy.trace(C_bar)))
print("IC_bar: "+str(numpy.trace(C_bar*J_bar**(-1))))

# Volumes
v = J_bar * V
print ("v:", v)

vs = dolfin.assemble(problem.kinematics.J * problem.dV)
print ("Vs0:", VS0)
print ("vs:", vs)
print ("Phi_s", vs/V)
vf = v - vs
print("v - vs:" +str(v - vs))
print("vf:" +str(vf))
print("porosity:"+str((V - VS0)/V))

# Macroscopic stress
print("sigma_tot_xx: " +str((dolfin.assemble(material.sigma[0,0] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[0,0])/v))
print("sigma_tot_xy: " +str((dolfin.assemble(material.sigma[0,1] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[0,1])/v))
print("sigma_tot_yx: " +str((dolfin.assemble(material.sigma[1,0] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[1,0])/v))
print("sigma_tot_yy: " +str((dolfin.assemble(material.sigma[1,1] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[1,1])/v))

print("sigma_s_xx: " +str((dolfin.assemble(material.sigma[0,0] * problem.kinematics.J * dV))/v))
print("sigma_s_xy: " +str((dolfin.assemble(material.sigma[0,1] * problem.kinematics.J * dV))/v))
print("sigma_s_xy: " +str((dolfin.assemble(material.sigma[1,0] * problem.kinematics.J * dV))/v))
print("sigma_s_yy: " +str((dolfin.assemble(material.sigma[1,1] * problem.kinematics.J * dV))/v))


print("sigma_f_xx: " +str((- vf * pf *dolfin.Identity(2)[0,0])/v))
print("sigma_f_xy: " +str((- vf * pf *dolfin.Identity(2)[0,1])/v))
print("sigma_f_xy: " +str((- vf * pf *dolfin.Identity(2)[1,0])/v))
print("sigma_f_yy: " +str((- vf * pf *dolfin.Identity(2)[1,1])/v))

#  numpy.linalg.det(problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))+numpy.eye(2))
# dolfin.det(dolfin.Identity(2) + dolfin.grad(problem.U_bar))

vf = v - vs

Vf0 = V - VS0

print("added fluid mass density: " +str((vf - Vf0)/V))

print("deformed porosity: " +str((vf)/v))

print("deformed surface: " +str(2 * math.pi * numpy.sqrt(vf/math.pi)))